In [38]:
final_model = "xlnet"

In [66]:
# Importing relevant libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

!pip install -q transformers
!pip install -q sentencepiece 
from transformers import BertTokenizer, XLNetTokenizerFast
from transformers import TFBertModel, XLNetModel
import sentencepiece

import pickle
import tensorflow as tf
from tensorflow import keras
import torch
from torch import nn
import torch.nn.functional as F

from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy

import os

# Global Variables
data_size = 234000
model_name = final_model + "-base-cased"

In [63]:
print("Downloading dataset")
!gdown --id 1bwE8nqeNx2-jNGq-DVkeoClrIWhFOJkm

Downloading...
From: https://drive.google.com/uc?id=1bwE8nqeNx2-jNGq-DVkeoClrIWhFOJkm
To: /content/treebank7.csv
8.43MB [00:00, 51.5MB/s]


In [64]:
df = pd.read_csv('/content/treebank7.csv')
df = df.iloc[:data_size]
phrases = np.array(df["phrases"])
sentiment_labels = np.array(df["labels"])

In [46]:
if final_model == "bert":
  tokenizer = BertTokenizer.from_pretrained(model_name)
elif final_model == "xlnet":
  tokenizer = XLNetTokenizerFast.from_pretrained(model_name)
else:
  print("Error: unrecognized model name")
max_token_len = np.max(np.array([len(tokenizer.encode(phrase)) for phrase in phrases]))
print(max_token_len)

76


In [47]:
# Encode dataset
token_ids = np.zeros(shape=(len(phrases), max_token_len), dtype=np.int32)
for index, phrase in enumerate(phrases):
  encoded = tokenizer.encode(phrase)
  token_ids[index][0 : len(encoded)] = encoded

In [48]:
sentiment_labels = np.array([round(value) for value in sentiment_labels])

In [77]:
# Split into training, testing and validation data
X_train, X_test, y_train, y_test = train_test_split(token_ids, sentiment_labels, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print(len(X_train), len(X_test), len(X_val))
print(len(y_train), len(y_test), len(y_val))

training_data_X = {"input_ids": torch.tensor(X_train), "attention_masks": torch.tensor((X_train != 0).astype(np.int32))}
testing_data_X = {"input_ids": torch.tensor(X_test), "attention_masks": torch.tensor((X_test != 0).astype(np.int32))}
validation_data_X = {"input_ids": torch.tensor(X_val), "attention_masks": torch.tensor((X_val != 0).astype(np.int32))}

140400 46800 46800
140400 46800 46800


In [68]:
if final_model == "bert":
  transformer_model = TFBertModel.from_pretrained(model_name)
elif final_model == "xlnet":
  transformer_model = XLNetModel.from_pretrained(model_name)
else:
  print("Error: model not found")

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [78]:
class TransformerModel(keras.Model):
  def __init__(self, dropout_prob = 0.1, **kwargs):
    super(TransformerModel, self).__init__()
    self.transformer = transformer_model
    self.dropout = Dropout(rate = dropout_prob)
    self.dense = Dense(1, activation="sigmoid")

  def call(self ,inputs, **kwargs):
    sequence_tokens, pooled_output = self.transformer(inputs, **kwargs).values()
    pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
    sentiment_label = self.dense(pooled_output)
    return sentiment_label

# class XLNetModel(nn.Module):
#   def __init__(self):
#     super(XLNetModel, self).__init__()
#     self.transformer = transformer_model
#     self.dropout = Dropout(p = 0.001)
#     self.dense = nn.Linear(1)

#   def forward(self, x):
#     x = self.transformer(x)
#     x = self.dropout(x)
#     return F.sigmoid(self.dense(x))

transformer_complete_model = TransformerModel()
transformer_complete_model.compile(optimizer=Adam(learning_rate=3e-5, epsilon=1e-08), loss="binary_crossentropy", metrics=["accuracy"])

In [79]:
history = transformer_complete_model.fit(x=training_data_X["input_ids"], y=y_train, epochs=5, batch_size=25, validation_data=(validation_data_X["input_ids"], y_val))

TypeError: ignored

In [74]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()